
# Day 16 AoC

🕎 [Day 16 description](https://adventofcode.com/2022/day/16) 🕎


## Setup

In [1]:
# imports
import os, re, sys, IPython, itertools, operator, functools, datetime, heapq, random

starttime = datetime.datetime.now()

In [2]:
# common helper, data import
def ans(val):
    return IPython.display.Markdown("**Answer: {}**".format(val))

data_fd = open('inputs/input-aoc-22-16.txt', 'r')
data = data_fd.read().strip().split('\n')

In [78]:
NODE_RE = re.compile(r'Valve ([A-Z]{2}) has flow rate=(\d+); tunnels? leads? to valves? (.*)')

def parse_node(line):
    m = NODE_RE.search(line)
    if m == None:
        print("Bad node: {}".format(line))
        return
    g = m.groups()
    return AOCNode(g[0], g[2], g[1])

class AOCNode(object):
    def __init__(self, name, linkstr, frate):
        self.name = name
        self._link_names = [x.strip() for x in linkstr.split(',')]
        self._linkstr = linkstr
        self._links = []
        self._frate = int(frate)
        
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return "{} ({}) -> {}".format(self.name, self._frate, "|".join(self._link_names))

    def update_links(self, name_map):
        for lname in self._link_names:
            self._links.append(name_map[lname])
    
    def flow_rate(self):
        return self._frate
    
    def linked_nodes(self):
        return iter(self._links)
    
    def __lt__(self, other):
        return self.name < other.name
    def __eq__(self, other):
        return self.name == other.name
    def __hash__(self):
        return hash(self.name)

def find_path_bfs(srcnode, tgtnode):
    found = set()
    path = []
    work_list = []
    for node in srcnode.linked_nodes():
        work_list.append((node,path[:]))
    while len(work_list) > 0:
        node, path = work_list.pop()
        path.append(node)
        if node == tgtnode:
            return path
        new_work = []
        for cnode in node.linked_nodes():
            if cnode not in found:
                new_work.append((cnode, path[:]))
                found.add(cnode)
        new_work.extend(work_list)
        work_list = new_work
    
class AOCVMap(object):
    def __init__(self, data):
        self._nodes = [ parse_node(l) for l in data ]
        self._name_map = {}
        for n in self._nodes:
            self._name_map[n.name] = n
        for node in self._nodes:
            node.update_links(self._name_map)
        self._start_node = self.find_node('AA')
        self._path_cache = {}
        
    def find_path(self, srcnode, tgtnode):
        if (srcnode, tgtnode) in self._path_cache:
            return self._path_cache[srcnode,tgtnode]
        p = find_path_bfs(srcnode, tgtnode)
        self._path_cache[srcnode,tgtnode] = p
        return p
            
    def find_node(self, name):
        return self._name_map[name]
    
    def start_node(self):
        return self._start_node
    def node_count(self):
        return len(self._nodes)
    
    def all_nodes(self):
        return iter(self._nodes)
    
class VolcanoPath(object):
    def __init__(self, map, starttime=30, log=True):
        self._map = map
        self.path = []
        self.action_list = []
        self.minutes = starttime
        self.pressure_release = 0
        self.total_release = 0
        self.released_nodes = set()
        self._log = log
        self._good_nodes = set([x for x in self._map.all_nodes() if x.flow_rate() > 0])
        
    def run(self):
        self.path.append(self._map.start_node())
        while self.minutes > 0:
            self.total_release += self.pressure_release
            self.minutes -= 1
            self.log_minute_start()
            act = self.choose_action()
            self.action_list.append(act)
            act.execute(self)
            self.log_minute_end()
        self.log_zero_minute()

    def current_node(self):
        return self.path[-1]
    
    def minutes_remaining(self):
        return self.minutes
    
    def good_nodes(self):
        # nodes that haven't been released and have flow > 0
        return self._good_nodes
        
    def release(self, node):
        assert node == self.path[-1], "Trying to release {} at {}".format(node, self.path[-1])
        assert node not in self.released_nodes, "Trying to re-release {}".format(node)
        self.pressure_release += node.flow_rate()
        self.released_nodes.add(node)
        self._good_nodes.remove(node)
        
    def move_to(self, node):
        assert node in self.path[-1]._links, "Bad move {} -> {}".format(self.path[-1], node)
        self.path.append(node)
    
    def choose_action(self):
        # Do nothing!
        return Action("NOOP", None)
    
    def log_minute_start(self):
        if not self._log:
            return
        display_minute = 30 - self.minutes
        
        print("== Minute {} ==".format(display_minute))
        
        if len(self.released_nodes) == 0:
            print("No Valves are open", end='')
        elif len(self.released_nodes) == 1:
            print("Valve {} is open".format(list(self.released_nodes)[0].name), end='')
        elif len(self.released_nodes) == 2:
            l = list(self.released_nodes)
            l.sort()
            print("Values {} and {} are open".format( l[0].name, l[1].name ), end='')
        else:
            l = list(self.released_nodes)
            l.sort()
            laststr = l.pop().name
            firststr = ", ".join([ n.name for n in l ]) 
            print("Valves {} and {} are open".format( firststr, laststr), end='')
        if self.pressure_release==0:
            print('.')
        else:
            print(', releasing {} pressure'.format(self.pressure_release))
            
    def log_minute_end(self):
        if not self._log:
            return        
        print(self.action_list[-1].log())
        
    def log_zero_minute(self):
        if not self._log:
            return
        print("You released a total of {} pressure.".format(self.total_release))

class ReplayPath(VolcanoPath):
    def __init__(self, map, replay):
        super().__init__(map)
        self._replay = replay[::-1]
        
    def choose_action(self):
        if len(self._replay):
            return self._replay.pop()
        else:
            return Action('NOOP', None)
    
class Action(object):
    def __init__(self, actionstr, tgt):
        self.actionstr = actionstr
        self.tgt = tgt

    def execute(self, path):
        if self.actionstr == 'MOVE':
            self._do_move(path)
        elif self.actionstr=='RELEASE':
            self._do_release(path)
            
    def _do_move(self, path):
        path.move_to(self.tgt)
        
    def _do_release(self, path):
        path.release(self.tgt)
        
    def log(self):
        if self.actionstr == "MOVE":
            return "You move to {}".format(self.tgt.name)
        elif self.actionstr == "RELEASE":
            return "You open valve {}".format(self.tgt.name)
        elif self.actionstr == "NOOP":
            return "You do nothing."
        else:
            return "You don't know what you're doing. What is going on?"

In [41]:
vmap = AOCVMap(data)



In [5]:
len( [ x for x in vmap.all_nodes() if x.flow_rate() > 0 ])

15

## Part 1

In [46]:
class BasicPath(VolcanoPath):
    def __init__(self, map):
        super().__init__(map)
        self._action_cache = []
        self._path_cache = {}
    
    def choose_action(self):
        if len(self._action_cache) == 0:
            self._next_action()
        if len(self._action_cache) > 0:
            return self._action_cache.pop()
        else:
            return Action('NOOP', None)
        
    def _goto_node_and_release(self, target):
        path_to_target = self._map.find_path(self.current_node(), target)
        self._action_cache = [ Action('MOVE', t) for t in path_to_target ]
        self._action_cache.append(Action('RELEASE', target))
        self._action_cache = self._action_cache[::-1]
        
        
class GreedyPath(BasicPath):
    
    def _calc_gain(self):
        all_gains = []
        for node in self.good_nodes():
            distance = len(self._map.find_path(self.current_node(), node))
            gain = node.flow_rate() * (self.minutes_remaining() - distance)
            all_gains.append((gain, node))
        all_gains.sort(key=lambda x: x[0])
        return all_gains
    
    def _next_action(self):
        nodes = self._calc_gain()
        if len(nodes) == 0:
            # nothing more to find!
            return
        gain,target = nodes.pop()
        self._goto_node_and_release(target)

In [71]:
class ExplicitPath(BasicPath):
    def __init__(self, map, pathchooser):
        super().__init__(map)
        self._chooser = pathchooser
        if hasattr(self._chooser, 'set_path'):
            self._chooser.set_path(self)
        
        
    def _next_action(self):
        try:
            target = next(self._chooser)
            self._goto_node_and_release(target)
        except StopIteration:
            self._action_cache = [ Action('NOOP', None) ]


In [73]:
class PathChooser(object):
    def __init__(self):
        self._path = None
        
    def set_path(self, path):
        self._path = path
        
    def __next__(self):
        return self._calc_gain()
    
    def _calc_gain(self):
        all_gains = []
        gn = list(self._path.good_nodes())
        for node in gn:
            distance = len(self._path._map.find_path(self._path.current_node(), node))
            gain = node.flow_rate() * max((self._path.minutes_remaining() - distance),1)
            all_gains.append(gain)
        return random.choices(gn, weights=all_gains)[0]


In [57]:
        nodes = self._good_nodes[:]
        random.shuffle(nodes)
        i = random.randint(1,len(nodes)-1)
        start = nodes[:i]
        end = nodes[i:]
        end.sort(key=lambda x:x.flow_rate())
        start.extend(end)
        nodes = start

NameError: name 'self' is not defined

In [77]:
class PathGenerator(object):
    def __init__(self, map, starttime=30):
        self._map = map
        self.best_path = None
        self.runtime = None
        self.path_count = 0
        self._good_nodes = [x for x in self._map.all_nodes() if x.flow_rate() > 0]
        self._starttime = starttime
        
    def generate(self):

        return ExplicitPath(self._map, PathChooser())
    
    def search_ipy(self, limit=None, update_seconds=10):
        starttime = datetime.datetime.now()
        last_update = starttime
        while limit == None or self.path_count < limit:
            new_path = self.generate()
            new_path._log  = False
            new_path._minutes = self._starttime
            new_path.run()
            
            if self.best_path == None:
                self.best_path = new_path
            elif new_path.total_release > self.best_path.total_release:
                self.best_path = new_path
            self.path_count += 1
            curtime = datetime.datetime.now()
            if last_update + datetime.timedelta(seconds=update_seconds) < curtime:
                last_update = curtime
                self.update_ipy(starttime, curtime)
    def update_ipy(self, starttime, curtime):
        IPython.display.clear_output(wait=True)
        runtime = curtime - starttime
        print("Runtime: {} | Found {} solutions | Best total pressure release {}".format(str(runtime), self.path_count, 
                                                                                         self.best_path.total_release))
        print("Processing at {} solutions per second".format(round(self.path_count/runtime.total_seconds())))

In [76]:
pg = PathGenerator(vmap)
pg.search_ipy()

Runtime: 0:16:20.013478 | Found 6678371 solutions | Best total pressure release 2330
Processing at 6815 solutions per second


KeyboardInterrupt: 

In [75]:
ep = ExplicitPath(vmap, PathChooser())
ep.run()

== Minute 1 ==
No Valves are open.
You move to WG
== Minute 2 ==
No Valves are open.
You move to TA
== Minute 3 ==
No Valves are open.
You move to PP
== Minute 4 ==
No Valves are open.
You move to QK
== Minute 5 ==
No Valves are open.
You open valve QK
== Minute 6 ==
Valve QK is open, releasing 24 pressure
You move to PQ
== Minute 7 ==
Valve QK is open, releasing 24 pressure
You move to JA
== Minute 8 ==
Valve QK is open, releasing 24 pressure
You move to VE
== Minute 9 ==
Valve QK is open, releasing 24 pressure
You move to KW
== Minute 10 ==
Valve QK is open, releasing 24 pressure
You move to VK
== Minute 11 ==
Valve QK is open, releasing 24 pressure
You open valve VK
== Minute 12 ==
Values QK and VK are open, releasing 46 pressure
You move to KW
== Minute 13 ==
Values QK and VK are open, releasing 46 pressure
You move to VE
== Minute 14 ==
Values QK and VK are open, releasing 46 pressure
You move to JA
== Minute 15 ==
Values QK and VK are open, releasing 46 pressure
You move to PQ
==

## Part 2

In [ ]:
class DualPath(object):
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.total_release = 0
        
    def run(self):
        pass
        
    

In [66]:
endtime = datetime.datetime.now()

print(endtime - starttime)

1:49:25.384020


## Notes


Naive solution takes about 6 minutes (`check_range`), revision that actually calculates ranges, ~20 seconds (`check_range2`).

## Bugs



1. Did not exclude beacons that existed in the count
2. was not returning the right set when calculating exclusion

## Test



In [ ]:
testdata = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II""".split('\n')

In [ ]:
tmap = AOCVMap(testdata)

In [ ]:
xcache = {}
xcache[tmap.find_node('AA'),tmap.find_node('GG')] = "Hithere"
xcache[(tmap.find_node('AA'),tmap.find_node('GG'))]

In [ ]:
actions = [
    Action('MOVE', tmap.find_node('DD')),
    Action('RELEASE', tmap.find_node('DD')),
    Action('MOVE', tmap.find_node('CC')),
    Action('MOVE', tmap.find_node('BB')),
    Action('RELEASE', tmap.find_node('BB')),
    Action('MOVE', tmap.find_node('AA')),
    Action('MOVE', tmap.find_node('II')),
    Action('MOVE', tmap.find_node('JJ')),
    Action('RELEASE', tmap.find_node('JJ')),
    Action('MOVE', tmap.find_node('II')),
    Action('MOVE', tmap.find_node('AA')),
    Action('MOVE', tmap.find_node('DD')),
    Action('MOVE', tmap.find_node('EE')),
    Action('MOVE', tmap.find_node('FF')),
    Action('MOVE', tmap.find_node('GG')),
    Action('MOVE', tmap.find_node('HH')),
    Action('RELEASE', tmap.find_node('HH')),
    Action('MOVE', tmap.find_node('GG')),
    Action('MOVE', tmap.find_node('FF')),
    Action('MOVE', tmap.find_node('EE')),
    Action('RELEASE', tmap.find_node('EE')),
    Action('MOVE', tmap.find_node('DD')),
    Action('MOVE', tmap.find_node('CC')),
    Action('RELEASE', tmap.find_node('CC')),
]



In [ ]:
tpath = ReplayPath(tmap, actions)

tpath.run()

In [ ]:
tgpath = GreedyPath(tmap)
tgpath.run()